<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon_Anmado-Is./blob/main/NDVI_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Earth Engine Python API 모듈 및 라이브러리 호출
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from osgeo import gdal
import requests
import json
import geopandas as gpd
import os
import shutil
import psutil
from google.colab import drive
import zipfile

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91')

더 세밀한 NDVI분석을 위해 임상도의 FRTP 코드를 기준으로 분류하여 트렌드를 살펴봅니다.

In [ ]:
# GeoJSON 파일 URL
url_anmado = 'https://github.com/jjangmo91/Cervus-nippon_Anmado-Is./raw/main/anmado.geojson'
url_songido = 'https://github.com/jjangmo91/Cervus-nippon_Anmado-Is./raw/main/songido.geojson'

# GeoJSON 파일을 GEE FeatureCollection으로 변환하는 함수
def geojson_to_feature_collection(url):
    response = requests.get(url)
    geojson = response.json()
    features = []
    for feature in geojson['features']:
        geom = ee.Geometry(feature['geometry'])
        feat = ee.Feature(geom, feature['properties'])
        features.append(feat)
    return ee.FeatureCollection(features)

# FeatureCollection 생성
anmado_fc = geojson_to_feature_collection(url_anmado)
songido_fc = geojson_to_feature_collection(url_songido)

# 각 frtp 코드별 AOI 설정 및 NDVI 분석
frtp_values = ['0', '1', '2', '3']  # 문자열로 변경
results_anmado = []
results_songido = []

for frtp_value in frtp_values:
    # 속성 이름을 FRTP_CD로 수정하여 필터링 (문자열로 필터링)
    filtered_anmado_fc = anmado_fc.filter(ee.Filter.eq('FRTP_CD', frtp_value))
    filtered_songido_fc = songido_fc.filter(ee.Filter.eq('FRTP_CD', frtp_value))

    # 필터링된 FeatureCollection 크기 확인
    count_anmado = filtered_anmado_fc.size().getInfo()
    count_songido = filtered_songido_fc.size().getInfo()

    print(f"FRTP_CD {frtp_value} - Anmado Feature Count: {count_anmado}")
    print(f"FRTP_CD {frtp_value} - Songido Feature Count: {count_songido}")

    if count_anmado == 0 or count_songido == 0:
        print(f"FRTP_CD {frtp_value}에 대한 FeatureCollection이 비어 있습니다.")
        continue  # 다음 FRTP_CD 값으로 넘어감

    # AOI 설정: 경계 상자가 아닌 원래의 geometry 사용
    aoi_anmado = filtered_anmado_fc.geometry()
    aoi_songido = filtered_songido_fc.geometry()

    results_anmado.append((frtp_value, aoi_anmado))
    results_songido.append((frtp_value, aoi_songido))

FRTP_CD 0 - Anmado Feature Count: 30
FRTP_CD 0 - Songido Feature Count: 8
FRTP_CD 1 - Anmado Feature Count: 32
FRTP_CD 1 - Songido Feature Count: 33
FRTP_CD 2 - Anmado Feature Count: 27
FRTP_CD 2 - Songido Feature Count: 30
FRTP_CD 3 - Anmado Feature Count: 31
FRTP_CD 3 - Songido Feature Count: 16


In [ ]:
# 색상 팔레트 설정 (FRTP_CD 값별로 색상을 지정)
colors = plt.cm.get_cmap('tab10', len(frtp_values)).colors  # 'tab10' 색상 팔레트 사용

# RGB 색상을 16진수 문자열로 변환하는 함수
def rgb_to_hex(rgb):
    return mcolors.to_hex(rgb)

# FRTP_CD 값에 따라 색상을 매핑
color_map = {frtp_value: rgb_to_hex(colors[idx]) for idx, frtp_value in enumerate(frtp_values)}

# 지도 객체 생성 및 AOI 시각화
Map = geemap.Map(center=[35.344031, 126.028111], zoom=12)

# 각 FRTP_CD에 대해 설정한 AOI를 시각화 (Anmado)
for frtp_value, aoi in results_anmado:
    if aoi is not None:
        Map.addLayer(aoi, {'color': color_map[frtp_value]}, f'Anmado AOI frtp {frtp_value}')

# 각 FRTP_CD에 대해 설정한 AOI를 시각화 (Songido)
for frtp_value, aoi in results_songido:
    if aoi is not None:
        Map.addLayer(aoi, {'color': color_map[frtp_value]}, f'Songido AOI frtp {frtp_value}')

# Anmado AOI 중심으로 설정
if results_anmado:
    Map.centerObject(results_anmado[0][1], 11)

# 지도 표시
Map

<ipython-input-5-46150416841c>:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap('tab10', len(frtp_values)).colors  # 'tab10' 색상 팔레트 사용


Map(center=[35.34933827590287, 126.01816083636925], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# 구름 및 그림자 마스킹 함수 정의
def maskLandsatClouds(image):
    # QA_PIXEL 레이어 선택
    qa = image.select('QA_PIXEL')

    # 비트 마스킹을 통해 구름(clouds), 구름 그림자(cloud shadows), 눈(snow) 식별
    cloud = qa.bitwiseAnd(1 << 5)
    cloudShadow = qa.bitwiseAnd(1 << 3)
    snow = qa.bitwiseAnd(1 << 4)

    # 구름, 그림자, 눈이 없는 맑은 상태의 이미지 마스크 생성
    mask = cloud.eq(0).And(cloudShadow.eq(0)).And(snow.eq(0))

    # 마스크를 적용하고, 스케일 조정을 위해 결과를 10000으로 나눔
    return image.updateMask(mask).divide(10000)

# 모든 FRTP_CD 값에 대해 개별적으로 이미지 컬렉션을 필터링하고 저장
landsat5_images_anmado_by_frtp = {}
landsat5_images_songido_by_frtp = {}
landsat8_images_anmado_by_frtp = {}
landsat8_images_songido_by_frtp = {}

for frtp_value, aoi_anmado in results_anmado:
    # Landsat 5 이미지 컬렉션 필터링 및 마스킹 적용
    landsat5_images_anmado_by_frtp[frtp_value] = (
        ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
        .filterDate("1985-01-01", "2012-05-05")
        .filterBounds(aoi_anmado)
        .map(maskLandsatClouds)
    )
    # Landsat 8 이미지 컬렉션 필터링 및 마스킹 적용
    landsat8_images_anmado_by_frtp[frtp_value] = (
        ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterDate("2013-04-11", "2024-01-31")
        .filterBounds(aoi_anmado)
        .map(maskLandsatClouds)
    )

for frtp_value, aoi_songido in results_songido:
    # Landsat 5 이미지 컬렉션 필터링 및 마스킹 적용
    landsat5_images_songido_by_frtp[frtp_value] = (
        ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
        .filterDate("1985-01-01", "2012-05-05")
        .filterBounds(aoi_songido)
        .map(maskLandsatClouds)
    )
    # Landsat 8 이미지 컬렉션 필터링 및 마스킹 적용
    landsat8_images_songido_by_frtp[frtp_value] = (
        ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterDate("2013-04-11", "2024-01-31")
        .filterBounds(aoi_songido)
        .map(maskLandsatClouds)
    )

# 여기서 각 이미지 컬렉션에 대한 추가 분석 및 NDVI 계산을 수행할 수 있습니다.

In [ ]:
# NDVI 계산 함수 정의 (Landsat 5)
def calculate_ndvi_landsat5(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')  # B4: NIR, B3: RED
    return image.addBands(ndvi)

# NDVI 계산 함수 정의 (Landsat 8)
def calculate_ndvi_landsat8(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  # B5: NIR, B4: RED
    return image.addBands(ndvi)

# NDVI 계산 적용 및 병합 (각 FRTP_CD 값에 대해)
all_ndvi_anmado_by_frtp = {}
all_ndvi_songido_by_frtp = {}

for frtp_value in frtp_values:
    if frtp_value in landsat5_images_anmado_by_frtp and frtp_value in landsat8_images_anmado_by_frtp:
        # Anmado 지역
        landsat5_images_anmado_ndvi = landsat5_images_anmado_by_frtp[frtp_value].map(calculate_ndvi_landsat5)
        landsat8_images_anmado_ndvi = landsat8_images_anmado_by_frtp[frtp_value].map(calculate_ndvi_landsat8)
        all_ndvi_anmado_by_frtp[frtp_value] = landsat5_images_anmado_ndvi.merge(landsat8_images_anmado_ndvi)

    if frtp_value in landsat5_images_songido_by_frtp and frtp_value in landsat8_images_songido_by_frtp:
        # Songido 지역
        landsat5_images_songido_ndvi = landsat5_images_songido_by_frtp[frtp_value].map(calculate_ndvi_landsat5)
        landsat8_images_songido_ndvi = landsat8_images_songido_by_frtp[frtp_value].map(calculate_ndvi_landsat8)
        all_ndvi_songido_by_frtp[frtp_value] = landsat5_images_songido_ndvi.merge(landsat8_images_songido_ndvi)

In [ ]:
import re

# 이미지 ID에서 날짜 추출하는 함수 (서버 측에서 실행)
def extract_date_from_id(image):
    img_id = image.get('system:index')
    date_str = ee.String(img_id).slice(-8)
    formatted_date = ee.Date.parse('yyyyMMdd', date_str).format('YYYY-MM-dd')
    return formatted_date

# NDVI 시계열 데이터를 추출하는 함수 정의
def calculate_ndvi_time_series(image, aoi):
    # 이미지에서 NDVI 밴드를 선택하고, 해당 AOI에 대해 NDVI의 평균을 계산
    mean_ndvi = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=30
    ).get('NDVI')

    # 이미지에서 날짜를 추출
    date = extract_date_from_id(image)

    # 결과를 Feature로 반환
    return ee.Feature(None, {'date': date, 'NDVI': mean_ndvi})

# AOI에 대한 NDVI 시계열 추출 (각 FRTP_CD 값에 대해)
ndvi_time_series_by_frtp = {}

for frtp_value in frtp_values:
    # Anmado 지역
    ndvi_timeseries_anmado = all_ndvi_anmado_by_frtp[frtp_value].map(
        lambda img: calculate_ndvi_time_series(img, results_anmado[frtp_values.index(frtp_value)][1])
    )

    # Songido 지역
    ndvi_timeseries_songido = all_ndvi_songido_by_frtp[frtp_value].map(
        lambda img: calculate_ndvi_time_series(img, results_songido[frtp_values.index(frtp_value)][1])
    )

    # 결과를 리스트로 변환
    ndvi_list_anmado = ndvi_timeseries_anmado.reduceColumns(ee.Reducer.toList(2), ['date', 'NDVI']).get('list').getInfo()
    ndvi_list_songido = ndvi_timeseries_songido.reduceColumns(ee.Reducer.toList(2), ['date', 'NDVI']).get('list').getInfo()

    # 데이터프레임 생성
    if len(ndvi_list_anmado) > 0 and len(ndvi_list_songido) > 0:
        dates_anmado, ndvi_values_anmado = zip(*ndvi_list_anmado)
        dates_songido, ndvi_values_songido = zip(*ndvi_list_songido)

        df_anmado = pd.DataFrame({'date': pd.to_datetime(dates_anmado), 'NDVI': ndvi_values_anmado})
        df_songido = pd.DataFrame({'date': pd.to_datetime(dates_songido), 'NDVI': ndvi_values_songido})

        # 날짜별로 중복된 값이 있을 경우 평균값으로 계산
        df_anmado_m = df_anmado.groupby('date', as_index=False).mean()
        df_songido_m = df_songido.groupby('date', as_index=False).mean()

        ndvi_time_series_by_frtp[frtp_value] = (df_anmado_m, df_songido_m)

        # NDVI 시계열 데이터 출력
        print(f"Anmado NDVI Time Series (FRTP_CD {frtp_value}, after averaging duplicates):")
        print(df_anmado_m.head())
        print(f"Songido NDVI Time Series (FRTP_CD {frtp_value}, after averaging duplicates):")
        print(df_songido_m.head())
    else:
        print(f"No valid NDVI data found for FRTP_CD {frtp_value} in Anmado or Songido.")

Anmado NDVI Time Series (FRTP_CD 0, after averaging duplicates):
        date      NDVI
0 1985-01-22  0.017604
1 1985-02-07  0.073579
2 1985-06-15  0.266947
3 1985-10-21  0.208662
4 1986-04-15  0.128983
Songido NDVI Time Series (FRTP_CD 0, after averaging duplicates):
        date      NDVI
0 1985-01-22  0.004238
1 1985-02-07  0.092792
2 1985-06-15  0.267634
3 1985-10-21  0.219395
4 1986-02-26  0.005685
Anmado NDVI Time Series (FRTP_CD 1, after averaging duplicates):
        date      NDVI
0 1985-01-22  0.063540
1 1985-02-07  0.102313
2 1985-06-15  0.300123
3 1985-10-21  0.239189
4 1986-04-15  0.140884
Songido NDVI Time Series (FRTP_CD 1, after averaging duplicates):
        date      NDVI
0 1985-01-22  0.014694
1 1985-02-07  0.129951
2 1985-06-15  0.319419
3 1985-10-21  0.234874
4 1986-02-26  0.006658
Anmado NDVI Time Series (FRTP_CD 2, after averaging duplicates):
        date      NDVI
0 1985-01-22  0.029497
1 1985-02-07  0.091114
2 1985-06-15  0.309370
3 1985-10-21  0.216351
4 1986